## ETL News

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='********-****-******-****-**********', project_access_token='p-************************************')
pc = project.project_context

In [4]:
%%capture
!pip install google-api-python-client==1.8.3
!pip install scrapy==2.1.0
!pip install attrs==19.3.0
!pip install scikit-learn==0.22
!pip install h5py==2.10.0
!pip install tensorflow==2.2.0rc0
!pip install keras==2.3.1
!pip install nltk==3.4

In [5]:
import pandas as pd

## Google Search API
from googleapiclient.discovery import build

### Crawler Scrapy
import unicodedata
import scrapy
import requests
from scrapy.http import TextResponse
import json
import string
from nltk import word_tokenize
from nltk.corpus import stopwords

### RNN Classifier
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn import preprocessing

import nltk
import unicodedata
import string
import re
import seaborn as sns
import itertools
import pickle

from nltk import word_tokenize
from nltk.corpus import stopwords
from keras.models import load_model

DIM_GLOVE = 50
n_words = 20000

Using TensorFlow backend.


In [6]:
## Google api
from googleapiclient.discovery import build


api_key = "*********************************"

my_api_key = "**********************************"
my_cse_id = "*******************************"  ## Custom Search element control API

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

In [7]:
dic_search = google_search("itaú", my_api_key, my_cse_id, num=10)
df_search = pd.DataFrame(list(dic_search.items())[5][1])

W0703 03:26:29.753084 139768582977344 __init__.py:46] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.l

In [8]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_09d659ed064a4c049d2d1b42b7c7853b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='********************************_*',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_*************************.get_object(Bucket='*********************-donotdelete-**-***********',Key='feature_news.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

feature_news = pd.read_csv(body)
filter_link = df_search[~df_search['link'].isin(feature_news.Link)].reset_index(drop=True)


In [9]:
## remove the quotes from the news
filter_itau_news = pd.DataFrame()
for i,x in enumerate(~filter_link['link'].str.contains('cotacoes')):   ### removing the quotation page information
    if x:
        filter_itau_news= filter_itau_news.append(df_search[i:i+1])  
filter_itau_news = filter_itau_news[['link','snippet','title','pagemap']]   ## filter cols in link, snippet and title
filter_itau_news = filter_itau_news.reset_index()
del filter_itau_news['index']

In [10]:
import requests
itau_news = pd.DataFrame(columns= ['Date','Link','Text'])
for i,x in enumerate(filter_itau_news['link']):
    res = requests.get(x)
    response = TextResponse(res.url, body=res.text, encoding='utf-8')
    lst = pd.DataFrame([[filter_itau_news['pagemap'][i]['metatags'][0]['article:published_time'][:10],filter_itau_news['link'][i], response.css(".article-content p::text").extract()]], columns= ['Date','Link','Text'])
    itau_news = itau_news.append(lst)

In [11]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
def strip_accents(sentence):
    try:
        sentence = unicode(sentence, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    sentence = unicodedata.normalize('NFD', sentence)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(sentence)

def remove_punction(text):
    no_punct = "".join(c for c in text if c not in string.punctuation)
    return no_punct

def remove_number(text):
    no_number = ''.join([i for i in text if not i.isdigit()])
    return no_number

def lower_caser(text):
    return text.lower()

def remove_stopwords(text):
        tokens = word_tokenize(text)
        tokens = [t for t in tokens if not t in stopwords.words('portuguese')]
        return " ".join(map(str.strip, tokens))

[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
itau_news['Text'] = itau_news['Text'].apply(remove_number)
itau_news['Text'] = itau_news['Text'].apply(remove_stopwords)
itau_news['Text'] = itau_news['Text'].apply(lower_caser)
itau_news['Text'] = itau_news['Text'].apply(strip_accents)
itau_news['Text'] = itau_news['Text'].apply(remove_punction)

In [15]:
from io import BytesIO
model = load_model(project.get_file('RNNModel.tflearn'))
train_le = preprocessing.LabelEncoder() # creating a labelencoder

/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [16]:
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_*****************************.get_object(Bucket='*********************-donotdelete-**-***********', Key='label_encoder.pkl')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 


In [17]:
from io import BytesIO
readrawdata = streaming_body_2.read()
label_encoder = pickle.load(BytesIO(readrawdata))
train_le.fit(label_encoder)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


LabelEncoder()

In [18]:
def RNN(Text):
    test_texts = [Text]
    tokenizer = Tokenizer(num_words=n_words)
    tokenizer.fit_on_texts(test_texts)
    test_sequences = tokenizer.texts_to_sequences(test_texts)
    test_input = pad_sequences(test_sequences, maxlen=DIM_GLOVE)
    test_predictions_probas = model.predict(test_input)
    test_predictions = test_predictions_probas.argmax(axis=-1)
    test_intent_predictions = train_le.inverse_transform(test_predictions)
    df_probas = pd.DataFrame(test_predictions_probas)
    df_intent = pd.DataFrame(test_intent_predictions)
    frames = [df_probas, df_intent]
    result = pd.concat(frames)
    result = pd.concat([df_probas, df_intent], axis=1)
    x = result.transpose()[0:104]     
    x.columns = ['probability']
    x = pd.to_numeric(x['probability'][:3])
    x = pd.DataFrame(x)
    labels = pd.DataFrame(label_encoder)
    labels = labels.reset_index()
    labels.columns = ['Labels','number']
    result_table= labels.join(x)
    result_table = result_table.sort_values(by=['probability'], ascending = False)
    result_table.head(5)
    return result_table.head(5).reset_index(drop = True)['number'][0]

In [19]:
itau_news['Class'] = itau_news['Text'].apply(RNN)

In [20]:
feature_news = feature_news.append(itau_news[['Date','Link','Class']]).reset_index(drop=True)

In [1]:
## Overwrite the csv
project.save_data(data=feature_news.to_csv(),file_name='itau_news.csv',overwrite=True)

NameError: name 'project' is not defined